In [12]:
import os
import pandas as pd
from Data_loader import Dataset_Custom
import argparse
import warnings
from tools import EarlyStopping
from torch.utils.data import Dataset, DataLoader
from utils import get_data
from Model import ANN
from torch import nn
import torch
import copy
from tqdm import tqdm
import random 
import numpy as np
warnings.filterwarnings("ignore")

In [13]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

seed_everything(0)

In [14]:
from Server import  Server
from Clients import Client
from Train import Trainer

In [15]:
parser_train = argparse.ArgumentParser(description='FL')
parser_train.add_argument('--root_path', type=str, default='../Data/GFC12/')
parser_train.add_argument('--dataset_paths', type=list, default=["wf1", "wf2", "wf3", "wf4", "wf5", "wf6", "wf7"])
parser_train.add_argument('--number_clients', type=int, default=7)
parser_train.add_argument('--seq_len', type=int, default=24*4)
parser_train.add_argument('--pred_len', type=int, default=6)
parser_train.add_argument('--label_len', type=int, default=0)
parser_train.add_argument('--train_length', type=int, default=16800)
parser_train.add_argument('--target', type=str, default='target')
parser_train.add_argument('--scale', type=bool, default=True)
parser_train.add_argument('--inverse', type=bool, default=True)

parser_train.add_argument('--lr', type=float, default=1e-4)
parser_train.add_argument('--global_epochs', type=int, default=200)
parser_train.add_argument('--local_epochs', type=int, default=1)
parser_train.add_argument('--fine_tune_epochs', type=int, default=20)
parser_train.add_argument('--patience', type=int, default=3)
parser_train.add_argument('--fed_patience', type=int, default=3)
parser_train.add_argument('--hidden_layers', type=list, default=[64,64,64])
parser_train.add_argument('--input_size', type=int, default=293)
parser_train.add_argument('--output_size', type=int, default=9)

parser_train.add_argument('--fine_tune_lr', type=float, default=1e-5)
parser_train.add_argument('--ensemble_flag', type=bool, default=True)
parser_train.add_argument('--batch_size', type=int, default=64)
parser_train.add_argument('--device', type=str, default='cuda:2' if torch.cuda.is_available() else 'cpu')
parser_train.add_argument('--forecasting_mode', type=str, default='prob')
parser_train.add_argument('--model_type', type=str, default='NN')
parser_train.add_argument('--model_save_path', type=str, default='../Model12/', help='location of model checkpoints')
parser_train.add_argument('--quantiles', type=list, default=[0.1,0.2,0.3,0.4, 0.5,0.6,0.7,0.8, 0.9])
args_train = parser_train.parse_args(args=[])


In [16]:
seed_everything(0)

In [17]:
clients=[]
for path in tqdm(args_train.dataset_paths):
    args_temp=copy.deepcopy(args_train)
    args_temp.dataset_paths=path
    clients.append(Client(args_temp))

  0%|          | 0/7 [00:00<?, ?it/s]

100%|██████████| 7/7 [00:05<00:00,  1.35it/s]


In [18]:
server = Server(args_train,clients)

In [9]:
'''import pickle
scale=6
# Load the server object
with open('../result/'+str(scale)+'/server_benchmark.pkl', 'rb') as f:
    server_benchmark = pickle.load(f)

# Load the clients object
with open('../result/'+str(scale)+'/clients_benchmark.pkl', 'rb') as f:
    clients_benchmark = pickle.load(f)


for i in range(7):
    clients[i].copy_client(clients_benchmark[i])

server = Server(args_train,clients)'''

In [20]:
server.fed_train()

test performance: [0.16784182537908424, 0.16830104068942267, 0.19582986071297567, 0.17998810240054783, 0.17237979622736369, 0.18448530691229317, 0.18157152966787554]
Epoch: 0 | Loss: 0.0861
Epoch: 0 | Loss: 0.0699
Epoch: 0 | Loss: 0.1111
Epoch: 0 | Loss: 0.0865
Epoch: 0 | Loss: 0.0781
Epoch: 0 | Loss: 0.0815
Epoch: 0 | Loss: 0.0901
Federated training Epoch [1/200] Val Loss: 0.0734
test performance: [0.07707778765016222, 0.08284751055379437, 0.09575462652599975, 0.08664964440546624, 0.08930565388745641, 0.08593025650471857, 0.09055326151827427]
Epoch: 0 | Loss: 0.0637
Epoch: 0 | Loss: 0.0701
Epoch: 0 | Loss: 0.0900
Epoch: 0 | Loss: 0.0747
Epoch: 0 | Loss: 0.0752
Epoch: 0 | Loss: 0.0757
Epoch: 0 | Loss: 0.0869
Federated training Epoch [2/200] Val Loss: 0.0676
test performance: [0.07154573047252959, 0.07818217344633112, 0.08699091715253379, 0.07997376558511224, 0.08321518137132468, 0.07858349449218135, 0.08346996527828582]
Epoch: 0 | Loss: 0.0562
Epoch: 0 | Loss: 0.0702
Epoch: 0 | Loss: 0

ANN(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=293, out_features=64, bias=True)
    (1-2): 2 x Linear(in_features=64, out_features=64, bias=True)
  )
  (relu): ReLU()
  (output): Linear(in_features=64, out_features=9, bias=True)
)

In [21]:
server.central_train()

Epoch [1/200] Train Loss: 0.0925 Val Loss: 0.0727
Epoch [2/200] Train Loss: 0.0715 Val Loss: 0.0656
Epoch [3/200] Train Loss: 0.0645 Val Loss: 0.0607
Epoch [4/200] Train Loss: 0.0609 Val Loss: 0.0593
Epoch [5/200] Train Loss: 0.0592 Val Loss: 0.0574
Epoch [6/200] Train Loss: 0.0581 Val Loss: 0.0566
Epoch [7/200] Train Loss: 0.0573 Val Loss: 0.0563
Epoch [8/200] Train Loss: 0.0568 Val Loss: 0.0551
Epoch [9/200] Train Loss: 0.0562 Val Loss: 0.0548
Epoch [10/200] Train Loss: 0.0557 Val Loss: 0.0544
Epoch [11/200] Train Loss: 0.0553 Val Loss: 0.0541
Epoch [12/200] Train Loss: 0.0550 Val Loss: 0.0538
Epoch [13/200] Train Loss: 0.0546 Val Loss: 0.0541
Epoch [14/200] Train Loss: 0.0542 Val Loss: 0.0530
Epoch [15/200] Train Loss: 0.0541 Val Loss: 0.0527
Epoch [16/200] Train Loss: 0.0537 Val Loss: 0.0526
Epoch [17/200] Train Loss: 0.0534 Val Loss: 0.0525
Epoch [18/200] Train Loss: 0.0533 Val Loss: 0.0525
Epoch [19/200] Train Loss: 0.0530 Val Loss: 0.0519
Epoch [20/200] Train Loss: 0.0525 Val Lo

ANN(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=293, out_features=64, bias=True)
    (1-2): 2 x Linear(in_features=64, out_features=64, bias=True)
  )
  (relu): ReLU()
  (output): Linear(in_features=64, out_features=9, bias=True)
)

In [22]:
server.local_train()

Launch Local Training!
Epoch [1/200] Train Loss: 0.0865 Val Loss: 0.0720
Epoch [2/200] Train Loss: 0.0700 Val Loss: 0.0640
Epoch [3/200] Train Loss: 0.0631 Val Loss: 0.0601
Epoch [4/200] Train Loss: 0.0602 Val Loss: 0.0580
Epoch [5/200] Train Loss: 0.0587 Val Loss: 0.0569
Epoch [6/200] Train Loss: 0.0576 Val Loss: 0.0564
Epoch [7/200] Train Loss: 0.0569 Val Loss: 0.0551
Epoch [8/200] Train Loss: 0.0562 Val Loss: 0.0561
Epoch [9/200] Train Loss: 0.0559 Val Loss: 0.0544
Epoch [10/200] Train Loss: 0.0555 Val Loss: 0.0539
Epoch [11/200] Train Loss: 0.0549 Val Loss: 0.0545
Epoch [12/200] Train Loss: 0.0550 Val Loss: 0.0533
Epoch [13/200] Train Loss: 0.0546 Val Loss: 0.0531
Epoch [14/200] Train Loss: 0.0541 Val Loss: 0.0531
Epoch [15/200] Train Loss: 0.0538 Val Loss: 0.0532
Epoch [16/200] Train Loss: 0.0535 Val Loss: 0.0530
Epoch [17/200] Train Loss: 0.0533 Val Loss: 0.0521
Epoch [18/200] Train Loss: 0.0530 Val Loss: 0.0530
Epoch [19/200] Train Loss: 0.0531 Val Loss: 0.0527
Epoch [20/200] Tr

In [23]:
local_fine_tune_losses=[]
local_fine_tune_preds=[]
local_fine_tune_models=[]
seed_everything(0)
for i in range(args_train.number_clients):
    local_fine_tune_pred,local_fine_tune_loss,local_fine_tune_model=clients[i].local_fine_tune()
    local_fine_tune_losses.append(local_fine_tune_loss)
    local_fine_tune_preds.append(local_fine_tune_pred)
    local_fine_tune_models.append(local_fine_tune_model)
print(local_fine_tune_losses)

1e-05 0
Epoch [1/20] Train Loss: 0.0529 Val Loss: 0.0517
Epoch [2/20] Train Loss: 0.0529 Val Loss: 0.0517
Epoch [3/20] Train Loss: 0.0528 Val Loss: 0.0516
Epoch [4/20] Train Loss: 0.0526 Val Loss: 0.0515
Epoch [5/20] Train Loss: 0.0526 Val Loss: 0.0515
Epoch [6/20] Train Loss: 0.0525 Val Loss: 0.0514
Epoch [7/20] Train Loss: 0.0525 Val Loss: 0.0514
Epoch [8/20] Train Loss: 0.0524 Val Loss: 0.0514
Epoch [9/20] Train Loss: 0.0524 Val Loss: 0.0514
Epoch [10/20] Train Loss: 0.0524 Val Loss: 0.0516
Epoch [11/20] Train Loss: 0.0523 Val Loss: 0.0513
Epoch [12/20] Train Loss: 0.0522 Val Loss: 0.0513
Epoch [13/20] Train Loss: 0.0522 Val Loss: 0.0512
Epoch [14/20] Train Loss: 0.0522 Val Loss: 0.0512
Epoch [15/20] Train Loss: 0.0521 Val Loss: 0.0512
Epoch [16/20] Train Loss: 0.0521 Val Loss: 0.0512
Epoch [17/20] Train Loss: 0.0520 Val Loss: 0.0512
Epoch [18/20] Train Loss: 0.0520 Val Loss: 0.0511
Epoch [19/20] Train Loss: 0.0520 Val Loss: 0.0511
Epoch [20/20] Train Loss: 0.0520 Val Loss: 0.0511
1

In [12]:
'''import utils import plot_prob_result
args_temp=copy.deepcopy(args_train)
args_temp.dataset_paths='wf7'
test_data, test_loader = get_data(args_train,flag='test')
actual_y=[]
for idx, (seq_x, seq_x_concat, seq_y) in enumerate(test_loader):
    actual_y.append(seq_y)
actual_y = torch.cat([torch.flatten(t) for t in actual_y])'''

"import utils import plot_prob_result\nargs_temp=copy.deepcopy(args_train)\nargs_temp.dataset_paths='wf7'\ntest_data, test_loader = get_data(args_train,flag='test')\nactual_y=[]\nfor idx, (seq_x, seq_x_concat, seq_y) in enumerate(test_loader):\n    actual_y.append(seq_y)\nactual_y = torch.cat([torch.flatten(t) for t in actual_y])"

In [24]:
fed_local_losses=[]
fed_local_preds=[]
fed_local_models=[]
for i in range(args_train.number_clients):
    fed_local_pred,fed_local_loss,fed_local_model=clients[i].fed_local_evaluation()
    fed_local_losses.append(fed_local_loss)
    fed_local_preds.append(fed_local_pred)
    fed_local_models.append(fed_local_model)
print(fed_local_losses)
print(np.mean(fed_local_losses))

[0.05583555196501212, 0.06445825300243212, 0.06835072321740732, 0.06237655860206036, 0.06335907018016258, 0.060239760326349166, 0.06499778458925143]
0.06280252884038215


In [25]:
local_losses=[]
local_preds=[]
local_models=[]
for i in range(args_train.number_clients):
    local_pred,local_loss,local_model=clients[i].local_evaluation()
    local_losses.append(local_loss)
    local_preds.append(local_pred)
    local_models.append(local_model)
print(local_losses)

[0.05624059526479407, 0.067097952398025, 0.06974936732129283, 0.06228208582057324, 0.06914129998688012, 0.06154689922520559, 0.06558624371146299]


In [26]:
central_losses=[]
central_preds=[]
central_models=[]
for i in range(args_train.number_clients):
    central_pred,central_loss,central_model=server.central_evaluation(dataset=i)
    central_losses.append(central_loss)
    central_preds.append(central_pred)
    central_models.append(central_model)
print(central_losses)

[0.05901773436607359, 0.07539156261729459, 0.08007056001982052, 0.06832268555355195, 0.07501274052599113, 0.06960178451127794, 0.07918094079431197]


In [30]:
print(local_losses)
print(central_losses)
print(fed_local_losses)
print(local_fine_tune_losses)

[0.05624059526479407, 0.067097952398025, 0.06974936732129283, 0.06228208582057324, 0.06914129998688012, 0.06154689922520559, 0.06558624371146299]
[0.05901773436607359, 0.07539156261729459, 0.08007056001982052, 0.06832268555355195, 0.07501274052599113, 0.06960178451127794, 0.07918094079431197]
[0.05583555196501212, 0.06445825300243212, 0.06835072321740732, 0.06237655860206036, 0.06335907018016258, 0.060239760326349166, 0.06499778458925143]
[0.05532612334554122, 0.0637155881228104, 0.0681344347635973, 0.06144858836770466, 0.06267848048257092, 0.06014074691354412, 0.06467975304168586]


In [28]:
df = pd.DataFrame({
    'local_losses': local_losses,
    'central_losses': central_losses,
    'fed_local_losses': fed_local_losses,
    'local_fine_tune_losses': local_fine_tune_losses
})
df.T.to_csv('losses.csv', index=False)

In [31]:
import pickle

# Save the server object
with open('../result/6/server_benchmark.pkl', 'wb') as f:
    pickle.dump(server, f)

# Save the clients object
with open('../result/6/clients_benchmark.pkl', 'wb') as f:
    pickle.dump(clients, f)

In [ ]:
import pickle

# Save server and clients
with open('server.pkl', 'wb') as f:
    pickle.dump(server, f)

with open('clients.pkl', 'wb') as f:
    pickle.dump(clients, f)

# Load server and clients
with open('server.pkl', 'rb') as f:
    server = pickle.load(f)

with open('clients.pkl', 'rb') as f:
    clients = pickle.load(f)